<a href="https://colab.research.google.com/github/pythonuzgit/elmurodov/blob/master/Image_classification_using_PyTorch_with_paddy_pests_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
kaggle datasets download -d zeeniye/paddy-pests-dataset

In [5]:
from zipfile import ZipFile
file_name = "/content/paddy-pests-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


Import all libraries

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
from PIL import Image


Clean data and remove non images files

In [7]:
path = '/content/paddy images'

for folder in os.listdir(path):
  for img_file in os.listdir(os.path.join(path, folder)):
    img_file = os.path.join(path, folder, img_file)

    try:
      img = Image.open(img_file)
      if img.mode != 'RGB':
        os.remove(img_file)

    except:
      os.remove(img_file)    


Data preprocessing and convert to tensor format

In [8]:
import torchvision
from torchvision import transforms

transform = transforms.Compose([
                                 transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(), 
                                 transforms.Normalize([0.5],
                                                      [0.5])
])

dataset = datasets.ImageFolder('/content/paddy images', transform = transform)

dataset_len = len(dataset)

train_len, test_len = dataset_len - 450, 450

train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

batch_size = 200

# train and test DataLoader

train_set = DataLoader(dataset = train_set, shuffle = True, batch_size=batch_size)
test_set = DataLoader(dataset = test_set, shuffle = True, batch_size=batch_size)

Checking the device

In [9]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
print(dev)

cuda


Build Model

In [12]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(p = 0.2)

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 4)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 12, kernel_size = 4)
    self.conv3 = nn.Conv2d(in_channels = 12, out_channels = 14, kernel_size = 4)
    self.conv4 = nn.Conv2d(in_channels = 14, out_channels = 16, kernel_size = 4)
    self.conv5 = nn.Conv2d(in_channels = 16, out_channels = 20, kernel_size = 4)

    
    self.fc1 = nn.Linear(in_features = 20 * 4 * 4, out_features = 250)
    self.fc2 = nn.Linear(in_features = 250, out_features = 200)
    self.fc3 = nn.Linear(in_features = 200, out_features = 50)
    self.fc4 = nn.Linear(in_features = 50, out_features = 10)
    self.fc5 = nn.Linear(in_features = 10, out_features = 2)



  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))
    x = self.pool(F.relu(self.conv5(x)))



    x = x.reshape(-1, 20 * 4 * 4)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = self.dropout(F.relu(self.fc4(x)))
    x = self.fc5(x)

    return x







net = Model().to(dev)

print(net)
   

Model(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(12, 14, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(14, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv5): Conv2d(16, 20, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=2, bias=True)
)


Define Loss and Optimizer functions

In [13]:
creterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001, weight_decay = 1e-5)

Train Data

In [17]:
net.train()

for epoch in range(15):
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set):
    inputs, labels = inputs.to(dev), labels.to(dev)
    output = net(inputs)
    output_idx = torch.argmax(output, dim = 1)
    total_correct += (labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = creterion(output, labels)
    running_loss += loss.item() * inputs.size(0)

    loss.backward()
    optimizer.step()



  print(f'Epoch: {epoch}  Loss: {running_loss/train_len}  Accuracy: {(total_correct/train_len) * 100}%') 




print('Finishing training')   

Epoch: 0  Loss: 0.6933500170707703  Accuracy: 46.24277456647399%
Epoch: 1  Loss: 0.6919451355934143  Accuracy: 73.98843930635837%
Epoch: 2  Loss: 0.6905965805053711  Accuracy: 76.878612716763%
Epoch: 3  Loss: 0.6892802715301514  Accuracy: 76.878612716763%
Epoch: 4  Loss: 0.6879176497459412  Accuracy: 76.878612716763%
Epoch: 5  Loss: 0.6873537302017212  Accuracy: 76.878612716763%
Epoch: 6  Loss: 0.6853795647621155  Accuracy: 76.878612716763%
Epoch: 7  Loss: 0.6825306415557861  Accuracy: 76.878612716763%
Epoch: 8  Loss: 0.674010694026947  Accuracy: 76.878612716763%
Epoch: 9  Loss: 0.6657476425170898  Accuracy: 76.878612716763%
Epoch: 10  Loss: 0.6405899524688721  Accuracy: 76.878612716763%
Epoch: 11  Loss: 0.6095064878463745  Accuracy: 75.72254335260115%
Epoch: 12  Loss: 0.5596941113471985  Accuracy: 76.878612716763%
Epoch: 13  Loss: 0.5148711204528809  Accuracy: 76.878612716763%
Epoch: 14  Loss: 0.555019736289978  Accuracy: 76.878612716763%
Finishing training


Test Data

In [19]:
with torch.no_grad():
  net.eval()
  total_loss = 0.0
  total_correct = 0.0
  for inputs, labels in test_set:
    labels = labels.to(dev)
    outputs = net(inputs.to(dev))
    loss = creterion(outputs, labels)
    total_loss += loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs, dim = 1)
    total_correct += sum(labels == output_idx)

  print(f'Accuracy: {(total_correct / test_len) * 100}%  Loss: {total_loss / test_len}')  

Accuracy: 79.77777862548828%  Loss: 0.48406195309427047


Save Training Model

In [20]:
torch.save(net.state_dict(), 'with_pests_vs_without_pits.pt')

Load saved model and test it on Test data

In [21]:
with torch.no_grad():
  model = Model().to(dev)
  model.load_state_dict(torch.load('/content/with_pests_vs_without_pits.pt'))

  net.eval()

  total_correct = 0.0

  for inputs, labels in test_set:
    labels = labels.to(dev)
    outputs = model(inputs.to(dev))
    output_idx = torch.argmax(outputs, dim =1)

    total_correct += sum(labels == output_idx)

  print(f'Accuracy: {(total_correct / test_len) * 100}%')  

Accuracy: 79.77777862548828%


In [23]:
img = Image.open('./00000012.JPG')

img = transform(img).unsqueeze(dim = 0).to(dev)
prediction = net(img)
print(torch.argmax(prediction))

tensor(1, device='cuda:0')
